# MAT281 - Laboratorio N°10



<a id='p1'></a>
## I.- Problema 01


<img src="https://www.goodnewsnetwork.org/wp-content/uploads/2019/07/immunotherapy-vaccine-attacks-cancer-cells-immune-blood-Fotolia_purchased.jpg" width="360" height="360" align="center"/>


El **cáncer de mama**  es una proliferación maligna de las células epiteliales que revisten los conductos o lobulillos mamarios. Es una enfermedad clonal; donde una célula individual producto de una serie de mutaciones somáticas o de línea germinal adquiere la capacidad de dividirse sin control ni orden, haciendo que se reproduzca hasta formar un tumor. El tumor resultante, que comienza como anomalía leve, pasa a ser grave, invade tejidos vecinos y, finalmente, se propaga a otras partes del cuerpo.

El conjunto de datos se denomina `BC.csv`, el cual contine la información de distintos pacientes con tumosres (benignos o malignos) y algunas características del mismo.


Las características se calculan a partir de una imagen digitalizada de un aspirado con aguja fina (FNA) de una masa mamaria. Describen las características de los núcleos celulares presentes en la imagen.
Los detalles se puede encontrar en [K. P. Bennett and O. L. Mangasarian: "Robust Linear Programming Discrimination of Two Linearly Inseparable Sets", Optimization Methods and Software 1, 1992, 23-34].


Lo primero será cargar el conjunto de datos:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
# cargar datos
df = pd.read_csv(os.path.join("data","BC.csv"), sep=",")
df['diagnosis'] = df['diagnosis'] .replace({'M':1,'B':0}) # target 
df.head()

Basado en la información presentada responda las siguientes preguntas:

1. [Realice un análisis exploratorio del conjunto de datos.](#e1)
2. [Normalizar las variables numéricas con el método **StandardScaler**.](#e2)
3. [Realizar un método de reducción de dimensionalidad visto en clases.](#e3)
4. [Aplique al menos tres modelos de clasificación distintos. Para cada uno de los modelos escogidos, realice una optimización de los hiperparámetros. además, calcule las respectivas métricas. Concluya.](#e4)




<a id='e1'></a>
**1.**

In [ ]:
# 1 Análisis exploratorio
# se usará esta funcion de la clase de eda
def resumen_por_columna(df,cols):
    pd_series = df[cols]
    
    # elementos distintos 
    l_unique = pd_series.unique()
    
    # elementos vacios
    
    l_vacios = pd_series[pd_series.isna()]
    
    df_info = pd.DataFrame({
        'columna': [cols],
        'unicos': [len(l_unique)],
        'vacios': [len(l_vacios)]
    })
    
    return df_info

In [ ]:
frames = []

for col in df.columns:
    aux_df = resumen_por_columna(df,col)
    frames.append(aux_df)
    
df_info = pd.concat(frames).reset_index(drop=True)
df_info['% vacios'] = df_info['vacios']/len(df)
df_info

In [ ]:
df.describe(include='all')

No hay datos vacíos (NaN). La columna 'diagnosis' es la que clasifica si el tumor es benigno (se definió marcar con 0) o maligno (marcar con 1)
Al ser variables numéricas el resto de las columnas presentan valores variados. Estas representan distintos parámetros acerca de las magnitudes de dimensiones (se refiere aquí a dimensiones físicas) de los tumores estudiados. Desde los nombres de estos parámetros, se puede inferir que podría ser plausible reducir la dimensionalidad del problema ya que por ejemplo el área de un tumor se puede calcular a partir de su radio o perímetro.

<a id='e2'></a>
**2**

In [ ]:
# 2
# se normalizan las variables numéricas
columns = df.drop(['id','diagnosis'], axis = 1).columns
dfn = df
dfn[columns] = StandardScaler().fit_transform(dfn[columns])
display(dfn.head())
display(dfn[columns].describe())

<a id='e3'></a>
**3**

In [ ]:
# 3
# Antes de elegir un método, se realiza un gráfico de correlación (ver Referencia)
dff = pd.DataFrame(df,columns=df.drop(['id','diagnosis'], axis = 1).columns)
corr = dff.corr()

# crear máscara para eliminar diagonal y celdas sobre la diagonal
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(17,17))

sns.heatmap(
    corr, # matriz de correlacion
    mask = mask, # aplicar mascara
    annot=True,  # imprimir numeros en celdas
    cbar_kws={"shrink": .5}, # achicar la leyenda de la barra de colores  
    ax = ax, # seleccionar ejes donde se ploteará
    fmt=".2f" # dar formato de dos decimales
)
a = ax.set_ylim(ax.get_ylim()[0] + 0.5, ax.get_ylim()[1]- 0.5) # arregla un bug que achicaba el eje y

Mientras mas claro el color hay mayor correlación entre las variables de las entradas. Mientras más oscuro menos relación.
Hay varios conjuntos con alta correlación, se pueden explicar algunas variables a partir de otras colineales y disminuir la dimensión del conjunto de datos a analizar.

En general las variables (promedio y worst) correspondientes al radio, perímetro y área de los núcleos presentan alta correlación entre sí. Se puede visualizar con scatterplot, por ejemplo, para los promedios de perímetro (**perimeter_mean**) y radio (**radius_mean**). Estas presentan una dispersión con tendencia lineal, deduciendo que se pueden asumir colineales, ayudando a disminuir la dimensionalidad del problema:

In [ ]:
sns.set(rc={'figure.figsize':(7,4)})

sns.scatterplot(x='perimeter_mean',
                y='radius_mean',
                hue = 'diagnosis',
                palette="Set1",
                data=df)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
x_training = dfn[columns]
y_training = dfn['diagnosis']
k = 10
columnas = list(x_training.columns.values)
seleccionadas = SelectKBest(f_classif, k=k).fit(x_training, y_training)

In [ ]:
catrib = seleccionadas.get_support()
atributos = [columnas[i] for i in list(catrib.nonzero()[0])]
atributos

In [ ]:
dfn[atributos]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from metrics_classification import summary_metrics

In [ ]:
#%%timeit
# Entrenamiento con todas las variables 
X = dfn[columns]
Y = dfn['diagnosis']

# split dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 2) 

# Creando el modelo
rlog = LogisticRegression()
rlog.fit(X_train, Y_train) # ajustando el modelo

predicciones = rlog.predict(X_test)

df_pred = pd.DataFrame({
    'y':Y_test,
    'yhat':predicciones
})
df_s1 = summary_metrics(df_pred).assign(name = 'Todas las variables')

In [ ]:
# Entrenamiento con las variables seleccionadas
X = dfn[atributos]
Y = dfn['diagnosis']

# split dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 2) 

# Creando el modelo
rlog = LogisticRegression()
rlog.fit(X_train, Y_train) # ajustando el modelo

predicciones = rlog.predict(X_test)

df_pred = pd.DataFrame({
    'y':Y_test,
    'yhat':predicciones
})

df_s2 = summary_metrics(df_pred).assign(name = 'Variables Seleccionadas')

In [ ]:
# juntar resultados en formato dataframe
pd.concat([df_s1,df_s2])

<a id='e4'></a>
**4**

In [ ]:
# 4 Optimización de hiperparametros
# Decision Tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor

# creación del modelo
model = DecisionTreeClassifier()
# rango de parametros
rango_criterion = ['gini','entropy']
rango_max_depth =np.array( [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150])
param_grid = dict(criterion=rango_criterion, max_depth=rango_max_depth)
param_grid
# aplicar greed search

gs = GridSearchCV(estimator=model, 
                  param_grid=param_grid, 
                  scoring='accuracy',
                  cv=5,
                  n_jobs=-1)

gs = gs.fit(X_train, Y_train)

In [ ]:
# imprimir resultados
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
# utilizando el mejor modelo
mejor_modelo = gs.best_estimator_
mejor_modelo.fit(X_train, Y_train)
print('Precisión: {0:.3f}'.format(mejor_modelo.score(X_test, Y_test)))

In [ ]:
df_dtc = pd.DataFrame(
    {
        'y':Y_test,
        'yhat': mejor_modelo.predict(X_test)
    }
)
metrics_temp1 = summary_metrics(df_dtc)
metrics_temp1['model'] = 'Decision Tree'

In [ ]:
# creación del modelo
# Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
# rango de parametros
rango_criterion = ['gini','entropy']
rango_max_depth =np.array( [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150])
param_grid = dict(criterion=rango_criterion, max_depth=rango_max_depth)
param_grid
# aplicar greed search

gs = GridSearchCV(estimator=model, 
                  param_grid=param_grid, 
                  scoring='accuracy',
                  cv=5,
                  n_jobs=-1)

gs = gs.fit(X_train, Y_train)

In [ ]:
# imprimir resultados
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
# utilizando el mejor modelo
mejor_modelo = gs.best_estimator_
mejor_modelo.fit(X_train, Y_train)
print('Precisión: {0:.3f}'.format(mejor_modelo.score(X_test, Y_test)))

In [ ]:
df_rfc = pd.DataFrame(
    {
        'y':Y_test,
        'yhat': mejor_modelo.predict(X_test)
    }
)
metrics_temp2 = summary_metrics(df_rfc)
metrics_temp2['model'] = 'Random Forest'

In [ ]:
# creación del modelo
# LogisticRegression
model = LogisticRegression()
# rango de parametros
rango_criterion = ['l1','l2']
rango_max_depth = np.array( [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150])
param_grid = dict(penalty=rango_criterion, C=rango_max_depth, solver=['liblinear'])
param_grid
# aplicar greed search

gs = GridSearchCV(estimator=model, 
                  param_grid=param_grid, 
                  scoring='accuracy',
                  cv=5,
                  n_jobs=-1)

gs = gs.fit(X_train, Y_train)

In [ ]:
# imprimir resultados
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
# utilizando el mejor modelo
mejor_modelo = gs.best_estimator_
mejor_modelo.fit(X_train, Y_train)
print('Precisión: {0:.3f}'.format(mejor_modelo.score(X_test, Y_test)))

In [ ]:
df_lr = pd.DataFrame(
    {
        'y':Y_test,
        'yhat': mejor_modelo.predict(X_test)
    }
)
metrics_temp3 = summary_metrics(df_lr)
metrics_temp3['model'] = 'Logistic Regression'

In [ ]:
df_metrics = pd.concat([metrics_temp1,metrics_temp2,metrics_temp3], ignore_index = True)
df_metrics

El modelo DecisionTreeClassifier presenta las mejores métricas en comparación a LogisticRegression y RandomForest. En general las métricas son elevadas, sin embargo al reducir la dimensionalidad a 10 columnas bajaron las métricas con respecto a no quitar columnas. Se podría haber iterado para logar obtener el numero óptimo de componentes que explican a las otras.

## Referencias: 

1. [Gráfico de correlación](https://sodocumentation.net/seaborn/topic/10634/correlation-plot)